In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 6.5 MB 39.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import torch
import transformers as ppb
import warnings
import gc
warnings.filterwarnings('ignore')

In [ ]:
def class_label(y):
  if y== 1.0:
    return 0 ;
  elif y== 1.5:
    return 1;
  elif y== 2.0:
    return 2;
  elif y== 2.5:
    return 3.0;
  elif y== 3:
    return 4;
  elif y== 3.5:
    return 5;
  elif y== 4.0:
    return 6;
  
def get_score_buckets(y_pred):
  if y_pred<1.5:
    return 1
  elif y_pred>1.25 and y_pred<=1.75:
    return 1.5
  elif y_pred>1.75 and y_pred<=2.25:
    return 2
  elif y_pred>2.25 and y_pred<=2.75:
    return 2.5
  elif y_pred>2.75 and y_pred<=3.25:
    return 3
  elif y_pred>3.25 and y_pred<3.5:
    return 3.5
  elif y_pred>=3.5:
    return 4

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/Colab Notebooks/3. Organisation Score Model Training/OS_data.xlsx" 
# df=pd.read_excel(path,sheet_name='Labels_data')
# all_icle_essay=pd.read_excel(path,sheet_name='All_Icle')
df=pd.read_excel(path,sheet_name='data')
df=df.fillna(0)
path_2=F"/content/gdrive/My Drive/Colab Notebooks/3. Organisation Score Model Training/train_test_5_fold.xlsx"

Mounted at /content/gdrive


In [ ]:
# 5 fold train test split using excel file 
fold_1=pd.read_excel(path_2,sheet_name='Fold1')
fold_1.fillna('')
fold_1_train=pd.merge(fold_1[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_1_test=(pd.merge(fold_1[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_1_train),len(fold_1_test))

fold_2=pd.read_excel(path_2,sheet_name='Fold2')
fold_2.fillna('')
fold_2_train=pd.merge(fold_2[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_2_test=(pd.merge(fold_2[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_2_train),len(fold_2_test))

fold_3=pd.read_excel(path_2,sheet_name='Fold3')
fold_3.fillna('')
fold_3_train=pd.merge(fold_3[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_3_test=(pd.merge(fold_3[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_3_train),len(fold_3_test))

fold_4=pd.read_excel(path_2,sheet_name='Fold4')
fold_4.fillna('')
fold_4_train=pd.merge(fold_4[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_4_test=(pd.merge(fold_4[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_4_train),len(fold_4_test))

fold_5=pd.read_excel(path_2,sheet_name='Fold5')
fold_5.fillna('')
fold_5_train=pd.merge(fold_5[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_5_test=(pd.merge(fold_5[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_5_train),len(fold_5_test))

799 201
800 200
799 201
801 199
801 199


For performance reasons, we'll only use 2,000 sentences from the dataset

We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# model_class, tokenizer_class, pretrained_weights = (ppb.BertForSequenceClassification, ppb.BertTokenizer, 'bert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
fold_1_train.iloc[:,0:17]

In [ ]:
def get_hidden_layer(batch_1):
  tokenized = batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=512,truncation=True)))  # changed from 300 to 512 for testing on uni server
  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  features = last_hidden_states[0][:,0,:].numpy()
  X_full = np.concatenate((batch_1.iloc[:, 15:].to_numpy(), features), axis = 1)
  return X_full

In [ ]:
train_features=get_hidden_layer(fold_1_train)
test_features=get_hidden_layer(fold_1_test)
train_labels=fold_1_train.Organisation_Score
test_labels=fold_1_test.Organisation_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_1_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score Train: ',rf.score(train_features, train_labels))
print('RF score Test: ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
svr = SVR(C=1000000)
svr.fit(train_features, train_labels)
svr_y_pred=svr.predict(test_features).round()
print('svr score Train: ',svr.score(train_features,train_labels))
print('svr score Test: ',svr.score(test_features,test_labels))
print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
print('MSE : ',mean_squared_error(test_labels, svr_y_pred))
gc.collect()

RF score Train:  0.9215582623150222
RF score Test:  0.4193029490070189
MAE :  0.3437810945273632
MSE :  0.19059701492537315
svr score Train:  0.9739621239613945
svr score Test:  0.2279966247370092
MAE :  0.39552238805970147
MSE :  0.30223880597014924


3678

In [ ]:
train_features=get_hidden_layer(fold_2_train)
test_features=get_hidden_layer(fold_2_test)
train_labels=fold_2_train.Organisation_Score
test_labels=fold_2_test.Organisation_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_2_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score Train: ',rf.score(train_features, train_labels))
print('RF score Test : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
svr = SVR(C=1000000)
svr.fit(train_features, train_labels)
svr_y_pred=svr.predict(test_features).round()
print('svr score Train: ',svr.score(train_features,train_labels))
print('svr score Test: ',svr.score(test_features,test_labels))
print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
print('MSE : ',mean_squared_error(test_labels, svr_y_pred))
gc.collect()

RF score Train:  0.9147736081219958
RF score Test :  0.5903788288288288
MAE :  0.31150000000000005
MSE :  0.15005000000000002
svr score Train:  0.9735305104583413
svr score Test:  0.421021354464559
MAE :  0.375
MSE :  0.305


96

In [ ]:
import joblib
joblib.dump(rf, "random_forest_org_train_batch_2.joblib")

['random_forest_org_train_batch_2.joblib']

In [ ]:
train_features=get_hidden_layer(fold_3_train)
test_features=get_hidden_layer(fold_3_test)
train_labels=fold_3_train.Organisation_Score
test_labels=fold_3_test.Organisation_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_3_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score Train: ',rf.score(train_features, train_labels))
print('RF score Test: ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
svr = SVR(C=1000000)
svr.fit(train_features, train_labels)
svr_y_pred=svr.predict(test_features).round()
print('svr score Train: ',svr.score(train_features,train_labels))
print('svr score Test: ',svr.score(test_features,test_labels))
print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
print('MSE : ',mean_squared_error(test_labels, svr_y_pred))
import gc
gc.collect()

RF score Train:  0.926332789803185
RF score Test:  0.3302312955386715
MAE :  0.3616915422885573
MSE :  0.21238805970149255
svr score Train:  0.9745438971176205
svr score Test:  0.10819947588835976
MAE :  0.39552238805970147
MSE :  0.3072139303482587


96

In [ ]:
train_features=get_hidden_layer(fold_4_train)
test_features=get_hidden_layer(fold_4_test)
train_labels=fold_4_train.Organisation_Score
test_labels=fold_4_test.Organisation_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_4_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score Train: ',rf.score(train_features, train_labels))
print('RF score Test: ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
svr = SVR(C=1000000)
svr.fit(train_features, train_labels)
svr_y_pred=svr.predict(test_features).round()
print('svr score Train: ',svr.score(train_features,train_labels))
print('svr score Test: ',svr.score(test_features,test_labels))
print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
print('MSE : ',mean_squared_error(test_labels, svr_y_pred))
gc.collect()

RF score Train:  0.9283403165010156
RF score Test:  0.32274321152924446
MAE :  0.37286432160804023
MSE :  0.23276381909547736
svr score Train:  0.973969544360033
svr score Test:  0.25500277805410143
MAE :  0.4221105527638191
MSE :  0.3316582914572864


96

In [ ]:
train_features=get_hidden_layer(fold_5_train)
test_features=get_hidden_layer(fold_5_test)
train_labels=fold_5_train.Organisation_Score
test_labels=fold_5_test.Organisation_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_5_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score Train: ',rf.score(train_features, train_labels))
print('RF score Test: ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
svr = SVR(C=1000000)
svr.fit(train_features, train_labels)
svr_y_pred=svr.predict(test_features).round()
print('svr score Train: ',svr.score(train_features,train_labels))
print('svr score Test: ',svr.score(test_features,test_labels))
print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
print('MSE : ',mean_squared_error(test_labels, svr_y_pred))
gc.collect()

RF score Train:  0.917336602263565
RF score Test:  0.5200081632653061
MAE :  0.3381909547738694
MSE :  0.18055276381909544
svr score Train:  0.9740161260603619
svr score Test:  0.30043427881479057
MAE :  0.4221105527638191
MSE :  0.3391959798994975


96

In [ ]:
print((fold_1_rf_score+fold_2_rf_score+fold_3_rf_score+fold_4_rf_score+fold_5_rf_score)/5)

0.34560558263956603


In [ ]:
# y_pred_bucket=pd.DataFrame(y_pred)
# y_pred_bucket.columns=['y_pred']
# y_pred_bucket['bucket']=y_pred_bucket.y_pred.apply(get_score_buckets)
# y_pred_numpy_bucket=y_pred_bucket[['bucket']].to_numpy()

In [ ]:
# for i in range(0,len(test_label_numpyarray)):
#   # if (y_test[i]==y_pred[i]) or (y_test[i]==y_pred[i]+1) or (y_test[i]==y_pred[i]-1):
#   # if (test_label_numpyarray[i]==y_pred[i]) or (test_label_numpyarray[i]==y_pred[i]+1):
#   if (test_label_numpyarray[i]==y_pred_numpy_bucket[i]) :
#     # print(i,': ',test_label_numpyarray[i],' : ',y_pred[i])
#     count = count + 1
#   total = total + 1
# accuracy = count/total*100
# print('RF regressor accuracy : ',accuracy)

In [ ]:
# from sklearn.metrics import confusion_matrix

# y_true=test_labels.apply(class_label)
# y_hat=y_pred_bucket.bucket.apply(class_label)
# confusion_matrix(y_true,y_hat,labels=[0,1,2,3,4,5,6])

# **svm performance analysis**

In [ ]:
# # exact score match for SVR

# from sklearn.svm import SVR
# from sklearn.metrics import mean_absolute_error,mean_squared_error
# svr = SVR(C=10000)
# svr.fit(test_features, test_labels)
# svr_y_pred=svr.predict(test_features).round()

# for i in range(0,len(test_label_numpyarray)):
#   # if (y_test[i]==y_pred[i]) or (y_test[i]==y_pred[i]+1) or (y_test[i]==y_pred[i]-1):
#   # if (test_label_numpyarray[i]==y_pred[i]) or (test_label_numpyarray[i]==y_pred[i]+1):
#   if (test_label_numpyarray[i]==svr_y_pred[i]):
#     count = count + 1
#   total = total + 1
# accuracy = count/total*100
# print('accuracy : ',accuracy)

# print('svr score : ',svr.score(test_features, test_labels))
# print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
# print('MSE : ',mean_squared_error(test_labels, svr_y_pred))

In [ ]:
# y_pred_bucket_svr=pd.DataFrame(svr_y_pred)
# y_pred_bucket_svr.columns=['y_pred_svr']
# y_pred_bucket_svr['bucket']=y_pred_bucket_svr.y_pred_svr.apply(get_score_buckets)
# y_pred_numpy_bucket_svr=y_pred_bucket_svr[['bucket']].to_numpy()
# # y_pred_bucket

In [ ]:
# y_true_svr=test_labels.apply(class_label)
# y_hat_svr=y_pred_bucket_svr.bucket.apply(class_label)
# confusion_matrix(y_true_svr,y_hat_svr,labels=[0,1,2,3,4,5,6])

# *Saving Trained model*

In [ ]:
import joblib
joblib.dump(rf, "random_forest_adu_features_org.joblib")
# loaded_rf = joblib.load("./random_forest.joblib")

Misc. sections

In [ ]:
# # Adding features for number of Sentence labels in each essay

# def no_of_prompt(essay):
#   prompt=essay.count('Prompt')
#   return prompt

# def no_of_Conclusion(essay):
#   Conclusion=essay.count('Conclusion')
#   return Conclusion
  
# def no_of_Elaboration(essay):
#   Elaboration=essay.count('Elaboration')
#   return Elaboration
  
# def no_of_MainIdea(essay):
#   MainIdea=essay.count('MainIdea')
#   return MainIdea

# def no_of_Rebuttal(essay):
#   Rebuttal=essay.count('Rebuttal')
#   return Rebuttal
# def no_of_Solution(essay):
#   Solution=essay.count('Solution')
#   return Solution

# def no_of_Suggestion(essay):
#   Suggestion=essay.count('Suggestion')
#   return Suggestion
  
# def no_of_Support(essay):
#   Support=essay.count('Support')
#   return Support

# def no_of_Thesis(essay):
#   Thesis=essay.count('Thesis')
#   return Thesis
  
# def no_of_Transition(essay):
#   Transition=essay.count('Transition')
#   return Transition

# def no_of_body(essay):
#   Body=essay.count('Body')
#   return Body

# def no_of_Introduction(essay):
#   Introduction=essay.count('Introduction')
#   return Introduction

# def no_of_None(essay):
#   None_count=essay.count('None')
#   return None_count
# def no_of_MajorClaim(essay):
#   MajorClaim=essay.count('MajorClaim')
#   return MajorClaim

# def no_of_Premise(essay):
#   Premise=essay.count('Premise')
#   return Premise

# def no_of_Claim(essay):
#   Claim=essay.count('Claim')
#   return Claim


In [ ]:
# df['Prompt']=df.Sentence_Label.apply(no_of_prompt)
# df['Elaboration']=df.Sentence_Label.apply(no_of_Elaboration)
# df['Conclusion']=df.Sentence_Label.apply(no_of_Conclusion)
# df['MainIdea']=df.Sentence_Label.apply(no_of_MainIdea)
# df['Rebuttal']=df.Sentence_Label.apply(no_of_Rebuttal)
# df['Solution']=df.Sentence_Label.apply(no_of_Solution)
# df['Suggestion']=df.Sentence_Label.apply(no_of_Suggestion)
# df['Support']=df.Sentence_Label.apply(no_of_Support)
# df['Thesis']=df.Sentence_Label.apply(no_of_Thesis)
# df['Transition']=df.Sentence_Label.apply(no_of_Transition)
# df['Introduction']=df.Paragraph_Label.apply(no_of_Introduction)
# df['Body']=df.Paragraph_Label.apply(no_of_body)
# df['Conclusion_Para']=df.Paragraph_Label.apply(no_of_Conclusion)
# df['None']=df.Annotated_ADU_Type.apply(no_of_None)
# df['MajorClaim']=df.Annotated_ADU_Type.apply(no_of_MajorClaim)
# df['Premise']=df.Annotated_ADU_Type.apply(no_of_Premise)
# df['Claim']=df.Annotated_ADU_Type.apply(no_of_Claim)

In [ ]:
df